Classifying sectors based on the the KeyBERT embedding of the repository description and the closest NAICS sector's description KeyBERT embedding. Distance between embeddings calculated using Euclidean distance and both strict and generous scoring approaches are explored.


In [ ]:
!pip install transformers
!pip install keybert
!pip install scikit-learn

import pandas as pd
import numpy as np
import os
from transformers import BertTokenizer, BertModel
from keybert import KeyBERT
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, log_loss, roc_auc_score


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.3 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.8.3-py3-none-any.whl size=39124 sha256=5e959513eedab98eb1fa66298e4d4606a191cf24d1895ff6d90684f23da32135
  Stored in directory: /root/.cache/pip/wheels/70/88/07/1a3bc11fd1dd5f89924a02dcbca89a3015e25e8faa31f904dc
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl 

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
data_path = "/content/drive/My Drive/Clubs/SAAS"
os.chdir(data_path)

In [ ]:
naics = pd.read_csv(data_path+"/NAICS descriptions.csv")
exam = pd.read_csv(data_path+"/testing_corpus_descriptions.csv")

In [ ]:
descriptions = naics['Descriptions'].tolist()
codes = naics['Sector'].tolist()

In [ ]:
correct = exam['Sector'].tolist()
readmes = exam['Description'].tolist()

In [ ]:
kw_model = KeyBERT(model='all-MiniLM-L6-v2')

In [ ]:
def embedding_generator(text):
  doc_embeddings, word_embeddings = kw_model.extract_embeddings(text)
  return doc_embeddings

In [ ]:
bible = {}

for idx,desc in enumerate(descriptions):
    desc_embedding = embedding_generator(desc)
    bible[codes[idx]] = desc_embedding

In [ ]:
def closest_sector(verse):
  closest_code = None
  closest_dist = float('inf')
  for codey, vek in bible.items():
    dist = np.linalg.norm(np.array(verse) - np.array(vek))
    print(codey, dist)
    if dist < closest_dist:
      closest_code = codey
      closest_dist = dist

  return closest_code

In [ ]:
def top_3_sectors(text):
  scores = {}
  dists = []
  for codey, vek in bible.items():
    dist = np.linalg.norm(embedding_generator(text) - vek)
    scores[dist] = codey
    dists.append(dist)
  dists = sorted(dists)
  first = scores[dists[0]]
  second = scores[dists[1]]
  third = scores[dists[2]]
  return first, second, third

In [ ]:
def testing():
  score = 0
  max_score = exam.shape[0]*4
  true_labels = []
  first_predicted_labels = []
  generous_predicted_labels = []
  for index, row in exam.iterrows():
    true_label = row['Sector']
    first, second, third = top_3_sectors(row['Description'])
    if true_label == first:
      score += 4
    elif true_label == second:
      score += 2
    elif true_label == third:
      score += 1
    true_labels.append(true_label)
    first_predicted_labels.append(first)
    if true_label == first or true_label==second or true_label==third:
      generous_predicted_labels.append(true_label)
    else:
      generous_predicted_labels.append(first)
  return true_labels, first_predicted_labels, generous_predicted_labels, score, max_score

In [ ]:
true_labels, first_predicted_labels, generous_predicted_labels, score, max_score = testing()

In [ ]:
print(f"The score is {score} out of {max_score}")

The score is 221 out of 376


In [ ]:
precision = precision_score(true_labels, first_predicted_labels, average="macro")
recall = recall_score(true_labels, first_predicted_labels, average="macro")
f1 = f1_score(true_labels, first_predicted_labels, average="macro")
print("For first_predicted_labels:")
print(f"Accuracy: {accuracy_score(true_labels, first_predicted_labels):.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1 score: {f1:.3f}")
#print(f"AUC-ROC: {roc_auc_score(true_labels, first_predicted_labels)}")

For first_predicted_labels:
Accuracy: 0.500
Precision: 0.533
Recall: 0.497
F1 score: 0.487


In [ ]:
precision = precision_score(true_labels, generous_predicted_labels, average="macro")
recall = recall_score(true_labels, generous_predicted_labels, average="macro")
f1 = f1_score(true_labels, generous_predicted_labels, average="macro")
print("For generous_predicted_labels:")
print(f"Accuracy: {accuracy_score(true_labels, generous_predicted_labels):.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1 score: {f1:.3f}")

For generous_predicted_labels:
Accuracy: 0.702
Precision: 0.731
Recall: 0.697
F1 score: 0.686
